In [1]:
urls = ["https://www.livemint.com/market/ipo/groww-ipo-day-1-gmp-date-review-subscription-status-other-details-apply-or-not-11762222226352.html"]

In [8]:
!pip install langchain langchain-google-genai langchain-community unstructured faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 59.5 MB/s eta 0:00:00


In [9]:
from langchain_community.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ","],
    chunk_size=1000,
    chunk_overlap=0
)

split_docs = text_splitter.split_documents(docs)

split_docs[0]

Document(metadata={'source': 'https://www.livemint.com/market/ipo/groww-ipo-day-1-gmp-date-review-subscription-status-other-details-apply-or-not-11762222226352.html'}, page_content='Search\n\nEnglish\n\nmint\n\nView Market Dashboard\n\nGroww IPO day 1: GMP, date, review, subscription status, other details. Apply or not?\n\nGroww IPO GMP today: According to market observers, shares of Billionbrains Garage Ventures Ltd are available at a premium of ₹17 in the grey market today\n\nAsit Manohar\n\nUpdated4 Nov 2025, 04:35 PM IST\n\nVideo thumbnail\n\nGroww IPO Day 1: The initial public offering (IPO) of Billionbrains Garage Ventures Ltd, popularly known as Groww, has opened today and will remain open until November 7, 2025. This means the Groww IPO subscription period will be from November 4 to 7, 2025. The Bengaluru-based fintech company has declared the Groww IPO price band at ₹95 to ₹100 per equity share. The Groww IPO is a Fresh Capital-cum-Offer for Sale, aiming to raise ₹6,632.30 cro

In [ ]:
GEMINI_API_KEY = "your api key"

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001", google_api_key=GEMINI_API_KEY)

In [13]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(split_docs, embeddings)

In [14]:
vectorstore.save_local("faiss_index")

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Load the vectorstore
loaded_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create a retriever
retriever = loaded_vectorstore.as_retriever()

# Initialize the Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", google_api_key=GEMINI_API_KEY)

In [20]:
llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7fcafd601a60>, default_metadata=(), model_kwargs={})

In [24]:
import langchain
query = "What is a price of a grow allotment date:?"

langchain.debug=True

In [25]:
from langchain.chains import RetrievalQA

# Create a retrieval chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Run the chain with the query
response = qa_chain.invoke(query)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is a price of a grow allotment date:?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What is a price of a grow allotment date:?",
  "context": "Search\n\nEnglish\n\nmint\n\nView Market Dashboard\n\nGroww IPO day 1: GMP, date, review, subscription status, other details. Apply or not?\n\nGroww IPO GMP today: According to market observers, shares of Billionbrains Garage Ventures Ltd are available at a premium of ₹17 in the grey market today\n\nAsit Manohar\n\nUpdated4 Nov 2025, 04:35 PM IST\n\nVideo thumbnail\n\nGroww IPO Day 1: The initial public offering (IPO) of Billionbrains Garage Ventures Ltd, popularly known as Groww, has opened today and will remain open until November 7, 2025. This means the Groww IPO subscription p

In [26]:
response

{'query': 'What is a price of a grow allotment date:?',
 'result': 'Based on the information provided, here are the details for the Groww IPO:\n\n*   **Price Band:** The Groww IPO price band is set at ₹95 to ₹100 per equity share.\n*   **Allotment Date:** The most likely date for the finalization of share allotment is November 8, 2025. It could also be November 10, 2025, in case of a delay.'}